In [4]:
import datetime
import gc
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import time
import warnings
import sklearn
from collections import Counter

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

from contextlib import contextmanager
from pandas.core.common import SettingWithCopyWarning
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold

warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

FEATS_EXCLUDED = ['first_active_month', 'target', 'card_id', 'outliers',
                  'hist_purchase_date_max', 'hist_purchase_date_min', 'hist_card_id_size',
                  'new_purchase_date_max', 'new_purchase_date_min', 'new_card_id_size',
                  'OOF_PRED', 'month_0']

/Users/jincui/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [1]:
import os
import ipyparallel as ipp
rc = ipp.Client()


In [2]:
rc.ids

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [5]:
hist_df = pd.read_hdf('Elo_fe.h5', 'hist_df')
new_merchant_df = pd.read_hdf('Elo_fe.h5', 'new_merchant_df')
#merchants.to_hdf('Elo_fe.h5', 'merchants')


In [22]:
### replace missing value (those without a merchant_id) with -1
hist_df['most_recent_sales_range'] = hist_df['most_recent_sales_range'].map({'A': 4, 'B': 3, 'C': 2, 'D': 1, 'E': 0, np.nan: -1}).astype(int)
hist_df['most_recent_purchases_range'] = hist_df['most_recent_purchases_range'].map({'A': 4, 'B': 3, 'C': 2, 'D': 1, 'E': 0, np.nan: -1}).astype(int)
new_merchant_df['most_recent_sales_range'] = new_merchant_df['most_recent_sales_range'].map({'A': 4, 'B': 3, 'C': 2, 'D': 1, 'E': 0, np.nan: -1}).astype(int)
new_merchant_df['most_recent_purchases_range'] = new_merchant_df['most_recent_purchases_range'].map({'A': 4, 'B': 3, 'C': 2, 'D': 1, 'E': 0, np.nan: -1}).astype(int)



In [72]:
fill_median = ['avg_sales_lag3', 'avg_sales_lag6', 'avg_sales_lag12',
             'avg_purchases_lag3', 'avg_purchases_lag6', 'avg_purchases_lag12',
             'active_months_lag3_orig', 'active_months_lag6_orig', 'active_months_lag12_orig']

for col in fill_median:
    hist_df[col] = hist_df[col].fillna(hist_df[col].median())
for col in fill_median:
    new_merchant_df[col] = new_merchant_df[col].fillna(new_merchant_df[col].median())

In [76]:
hist_df.isna().sum().sum()

0

In [63]:
fill_zero = ['state_id', 'subsector_id', 'merchant_group_id',
            'numerical_1', 'numerical_2', 'numerical_ratio']

fill_minusone = ['mer_category_1', 'mer_category_2_1.0', 'mer_category_2_2.0',
                 'mer_category_2_3.0', 'mer_category_2_4.0', 'mer_category_2_5.0', 
                 'mer_category_2_nan', 'has_last_month_transaction', 
                 'has_last3_months_revenue', 'category_4',
                 'active_months_lag3_1', 'active_months_lag3_2', 'active_months_lag3_3',
                 'active_months_lag6_1', 'active_months_lag6_2', 'active_months_lag6_3',
                 'active_months_lag6_4', 'active_months_lag6_5', 'active_months_lag6_6',
                 'active_months_lag12_1', 'active_months_lag12_2', 'active_months_lag12_3',
                 'active_months_lag12_4', 'active_months_lag12_5', 'active_months_lag12_6',
                 'active_months_lag12_7', 'active_months_lag12_8', 'active_months_lag12_9',
                 'active_months_lag12_10', 'active_months_lag12_11', 'active_months_lag12_12']

fill_median = ['avg_sales_lag3', 'avg_sales_lag6', 'avg_sales_lag12',
             'avg_purchases_lag3', 'avg_purchases_lag6', 'avg_purchases_lag12',
             'active_months_lag3_orig', 'active_months_lag6_orig', 'active_months_lag12_orig']

hist_df[fill_zero] = hist_df[fill_zero].fillna(0)
hist_df[fill_minusone] = hist_df[fill_minusone].fillna(-1)
for col in fill_median:
    hist_df[col] = hist_df[col].fillna(hist_df[col].median())
    
new_merchant_df[fill_zero] = new_merchant_df[fill_zero].fillna(0)
new_merchant_df[fill_minusone] = new_merchant_df[fill_minusone].fillna(-1)
for col in fill_median:
    new_merchant_df[col] = new_merchant_df[col].fillna(new_merchant_df[col].median())

In [24]:
## Attention that new_merchant_df authorized_flag are all Yes!!!! so we can remove them
new_merchant_df = new_merchant_df.drop(['authorized_flag'], axis=1)

In [9]:
hist_df['price'] = hist_df['purchase_amount']/hist_df['installments_orig']
hist_df['price'][hist_df['price']>1] = 150000
hist_df['price'][hist_df['price']<0] = -150000
hist_df['price'][hist_df['installments_orig']==-1] = 0
hist_df['price'][hist_df['installments_orig']==999] = 0
hist_df['price'] = hist_df['price']/150000

new_merchant_df['price'] = new_merchant_df['purchase_amount']/new_merchant_df['installments_orig']
new_merchant_df['price'][new_merchant_df['price']>1] = 75
new_merchant_df['price'][new_merchant_df['price']<0] = -75
new_merchant_df['price'][new_merchant_df['installments_orig']==-1] = 0
new_merchant_df['price'][new_merchant_df['installments_orig']==999] = 0
new_merchant_df['price'] = new_merchant_df['price']/75


KeyboardInterrupt: 

In [168]:
### save for quicker access
hist_df.to_hdf('Elo_fe.h5', 'hist_df_fillna')
new_merchant_df.to_hdf('Elo_fe.h5', 'new_merchant_df_fillna')

In [104]:
hist_df.to_hdf('Elo_fe.h5', 'hist_df_fillna_table', format='table')
new_merchant_df.to_hdf('Elo_fe.h5', 'new_merchant_df_fillna_table', format='table')


### read_hdf

In [10]:
hist_df = pd.read_hdf('Elo_fe.h5', 'hist_df_fillna')
new_merchant_df = pd.read_hdf('Elo_fe.h5', 'new_merchant_df_fillna')

In [25]:
print(hist_df.shape)
print(new_merchant_df.shape)
hist_df.head()

(29208684, 119)
(1969039, 106)


,authorized_flag,card_id,city_id,category_1,merchant_category_id,merchant_id,purchase_amount,state_id,subsector_id,merchant_group_id,numerical_1,numerical_2,mer_category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,avg_purchases_lag3,avg_sales_lag6,avg_purchases_lag6,avg_sales_lag12,avg_purchases_lag12,category_4,numerical_ratio,active_months_lag3_orig,active_months_lag6_orig,active_months_lag12_orig,active_months_lag3_1,active_months_lag3_2,active_months_lag3_3,active_months_lag6_1,active_months_lag6_2,active_months_lag6_3,active_months_lag6_4,active_months_lag6_5,active_months_lag6_6,active_months_lag12_1,active_months_lag12_2,active_months_lag12_3,active_months_lag12_4,active_months_lag12_5,active_months_lag12_6,active_months_lag12_7,active_months_lag12_8,active_months_lag12_9,active_months_lag12_10,active_months_lag12_11,active_months_lag12_12,mer_category_2_1.0,mer_category_2_2.0,mer_category_2_3.0,mer_category_2_4.0,mer_category_2_5.0,mer_category_2_nan,has_last_month_transaction,has_last3_months_revenue,purchase_date_year,purchase_date_month,purchase_date_day,purchase_date_dayofweek,purchase_date_hour,purchase_date_weekofyear,purchase_date_weekday,installments_orig,month_lag_orig,month_diff_orig,purchase_date_quarter,month_lag_-13,month_lag_-12,month_lag_-11,month_lag_-10,month_lag_-9,month_lag_-8,month_lag_-7,month_lag_-6,month_lag_-5,month_lag_-4,month_lag_-3,month_lag_-2,month_lag_-1,month_lag_0,month_diff_11,month_diff_12,month_diff_13,month_diff_14,month_diff_15,month_diff_16,month_diff_17,month_diff_18,month_diff_19,month_diff_20,month_diff_21,month_diff_22,month_diff_23,month_diff_24,installments_-1,installments_0,installments_1,installments_2,installments_3,installments_4,installments_5,installments_6,installments_7,installments_8,installments_9,installments_10,installments_11,installments_12,installments_999,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_2_nan,category_3_A,category_3_B,category_3_C,category_3_nan
0,1,C_ID_4e6213e9bc,88,0,80,M_ID_e020e9b302,-0.703331,16,37,35.0,23.470597,23.014512,0.0,4,4,1.08,1.082451,1.14,1.114135,1.19,1.156844,1.0,1.019817,3.0,6.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2017,6,25,6,15,25,1,0,-8,11,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
1,1,C_ID_4e6213e9bc,88,0,367,M_ID_86ec983688,-0.733128,16,16,2084.0,1.092658,-0.057471,0.0,4,4,1.06,1.052071,1.06,1.058605,1.05,1.062087,1.0,-19.012457,3.0,6.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2017,7,15,5,12,28,1,0,-7,11,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
2,1,C_ID_4e6213e9bc,88,0,80,M_ID_979ed661fc,-0.720386,16,37,27369.0,0.021849,0.021849,0.0,2,2,0.98,0.974653,0.98,0.967058,0.97,0.956668,1.0,1.000000,3.0,6.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2017,8,9,2,22,32,0,0,-6,11,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
3,1,C_ID_4e6213e9bc,88,0,560,M_ID_e6d5ae8ea6,-0.735352,16,34,24104.0,-0.057471,-0.057471,0.0,1,2,1.00,1.053443,0.88,0.897406,0.86,0.864394,1.0,1.000000,3.0,6.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2017,9,2,5,10,35,1,0,-5,12,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
4,1,C_ID_4e6213e9bc,88,0,80,M_ID_e020e9b302,-0.722865,16,37,35.0,23.470597,23.014512,0.0,4,4,1.08,1.082451,1.14,1.114135,1.19,1.156844,1.0,1.019817,3.0,6.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2017,3,10,4,1,10,0,0,-11,12,0,0,0,1,0,0,

In [6]:
def Range(x):
    res = max(x) - min(x)
    return res

def num_zero(x):
    return len(x[x==0])

def num_one(x):
    return len(x[x==1])

def num_minusone(x):
    return len(x[x==-1])

def num_triplenine(x):
    return len(x[x==999])
 
def num_more_than_one(x):
    return len(x[x>1])

def num_less_than_one(x):
    return len(x[x<1])

def sumsq(x):
    return sum(x**2)

def ratio_one(x):
    return len(x[x==1.0])/len(x)

def ratio_zero(x):
    return len(x[x==0.0])/len(x)

def ratio_minusone(x):
    return len(x[x==-1.0])/len(x)

def ratio_triplenine(x):
    return len(x[x==999])/len(x)

def ratio_more_than_one(x):
    return len(x[x>1.0])/len(x)

def ratio_less_than_one(x):
    return len(x[x<1.0])/len(x)

def complexity(x):
    counts = np.array(list(Counter(x).values()))
    S = sum(counts)
    counts = counts/S
    return sumsq(counts)

def Mode(x):
    # if the most frequent & 2nd most frequent one have the same frequencies, take the bigger one
    counts = Counter(x)
    tup = sorted(counts.items(), key=lambda kv: kv[1])
    return tup[-1][0]

def mode(x):
    # take average of the 2
    counts = Counter(x)
    tup = sorted(counts.items(), key=lambda kv: kv[1])
    try:
        assert tup[-1][1] == tup[-2][1]
        return (tup[-1][d0] + tup[-2][0])/2.0
    except Exception:
        return tup[-1][0]


In [9]:
aggs = {}
aggs['merchant_id'] = ['count', 'nunique', complexity]
aggs['purchase_amount'] = ['count', 'nunique', complexity, 'mean', Mode]

new_merchant_df.iloc[:10000,:].groupby(['card_id', 'purchase_date_month']).agg(aggs)


merchant_id                     \
                                          count nunique complexity   
card_id         purchase_date_month                                  
C_ID_006005dd0a 3.0                           1       1   1.000000   
                4.0                           3       3   0.333333   
C_ID_008b129f22 3.0                           2       2   0.500000   
                4.0                           4       4   0.250000   
                16.0                          2       2   0.500000   
C_ID_01dc6f675a 3.0                           4       4   0.250000   
                4.0                           6       6   0.166667   
                15.0                          1       1   1.000000   
                16.0                          1       1   1.000000   
C_ID_02527da786 2.0                           9       9   0.111111   
                3.0                           5       5   0.200000   
                14.0                          3       3   0.333333   
                15.0                          1       1   1.000000   
C_ID_0261d84268 2.0                           6       6   0.166667   
                3.0                           2       2   0.500000   
                14.0                          1       1   1.000000   
C_ID_02950a95bb 3.0                           1       1   1.000000   
                4.0                           1       1   1.000000   
C_ID_0339c7acac 4.0                           2       2   0.500000   
                15.0                          2       2   0.500000   
                16.0                          3       3   0.333333   
C_ID_039d398077 10.0                         14      14   0.071429   
                11.0                          9       9   0.111111   
                22.0                          3       3   0.333333   
                23.0                          2       2   0.500000   
C_ID_044d27650c 1.0                           3       3   0.333333   
                2.0                           5       5   0.200000   
C_ID_04c6befcde 3.0                           6       6   0.166667   
                4.0                           6       6   0.166667   
C_ID_051ce6e8f3 2.0                           1       1   1.000000   
                3.0                           4       4   0.250000   
                14.0                          1       1   1.000000   
C_ID_05242bb8c4 3.0                           5       5   0.200000   
                4.0                           3       3   0.333333   
                15.0                          2       2   0.500000   
                16.0                          1       1   1.000000   
C_ID_05fce29cca 1.0                           3       3   0.333333   
                2.0                           1       1   1.000000   
                13.0                          2       2   0.500000   
C_ID_0682c61725 3.0                           1       1   1.000000   
                4.0                           1       1   1.000000   
C_ID_06ba2a355d 3.0                           4       4   0.250000   
                4.0                           4       4   0.250000   
C_ID_0720bc334a 3.0                           8       8   0.125000   
C_ID_0848be1196 2.0                           1       1   1.000000   
C_ID_086fe1da99 3.0                           9       9   0.111111   
                4.0                           6       6   0.166667   
                15.0                          4       4   0.250000   
                16.0                          3       3   0.333333   
C_ID_08a8702de2 3.0                          13      13   0.076923   
...                                         ...     ...        ...   
C_ID_fcec2b5dc7 16.0                          1       1   1.000000   
C_ID_fd212fdea8 10.0                          1       1   1.000000   
                11.0                         11      11   0.090909   
                23.0                          

In [26]:
hist_df.head(20)

,authorized_flag,card_id,city_id,category_1,merchant_category_id,merchant_id,purchase_amount,state_id,subsector_id,merchant_group_id,numerical_1,numerical_2,mer_category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,avg_purchases_lag3,avg_sales_lag6,avg_purchases_lag6,avg_sales_lag12,avg_purchases_lag12,category_4,numerical_ratio,active_months_lag3_orig,active_months_lag6_orig,active_months_lag12_orig,active_months_lag3_1,active_months_lag3_2,active_months_lag3_3,active_months_lag6_1,active_months_lag6_2,active_months_lag6_3,active_months_lag6_4,active_months_lag6_5,active_months_lag6_6,active_months_lag12_1,active_months_lag12_2,active_months_lag12_3,active_months_lag12_4,active_months_lag12_5,active_months_lag12_6,active_months_lag12_7,active_months_lag12_8,active_months_lag12_9,active_months_lag12_10,active_months_lag12_11,active_months_lag12_12,mer_category_2_1.0,mer_category_2_2.0,mer_category_2_3.0,mer_category_2_4.0,mer_category_2_5.0,mer_category_2_nan,has_last_month_transaction,has_last3_months_revenue,purchase_date_year,purchase_date_month,purchase_date_day,purchase_date_dayofweek,purchase_date_hour,purchase_date_weekofyear,purchase_date_weekday,installments_orig,month_lag_orig,month_diff_orig,purchase_date_quarter,month_lag_-13,month_lag_-12,month_lag_-11,month_lag_-10,month_lag_-9,month_lag_-8,month_lag_-7,month_lag_-6,month_lag_-5,month_lag_-4,month_lag_-3,month_lag_-2,month_lag_-1,month_lag_0,month_diff_11,month_diff_12,month_diff_13,month_diff_14,month_diff_15,month_diff_16,month_diff_17,month_diff_18,month_diff_19,month_diff_20,month_diff_21,month_diff_22,month_diff_23,month_diff_24,installments_-1,installments_0,installments_1,installments_2,installments_3,installments_4,installments_5,installments_6,installments_7,installments_8,installments_9,installments_10,installments_11,installments_12,installments_999,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_2_nan,category_3_A,category_3_B,category_3_C,category_3_nan
0,1,C_ID_4e6213e9bc,88,0,80,M_ID_e020e9b302,-0.703331,16,37,35.0,23.470597,23.014512,0.0,4,4,1.08,1.082451,1.14,1.114135,1.19,1.156844,1.0,1.019817,3.0,6.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2017,6,25,6,15,25,1,0,-8,11,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
1,1,C_ID_4e6213e9bc,88,0,367,M_ID_86ec983688,-0.733128,16,16,2084.0,1.092658,-0.057471,0.0,4,4,1.06,1.052071,1.06,1.058605,1.05,1.062087,1.0,-19.012457,3.0,6.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2017,7,15,5,12,28,1,0,-7,11,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
2,1,C_ID_4e6213e9bc,88,0,80,M_ID_979ed661fc,-0.720386,16,37,27369.0,0.021849,0.021849,0.0,2,2,0.98,0.974653,0.98,0.967058,0.97,0.956668,1.0,1.000000,3.0,6.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2017,8,9,2,22,32,0,0,-6,11,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
3,1,C_ID_4e6213e9bc,88,0,560,M_ID_e6d5ae8ea6,-0.735352,16,34,24104.0,-0.057471,-0.057471,0.0,1,2,1.00,1.053443,0.88,0.897406,0.86,0.864394,1.0,1.000000,3.0,6.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2017,9,2,5,10,35,1,0,-5,12,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
4,1,C_ID_4e6213e9bc,88,0,80,M_ID_e020e9b302,-0.722865,16,37,35.0,23.470597,23.014512,0.0,4,4,1.08,1.082451,1.14,1.114135,1.19,1.156844,1.0,1.019817,3.0,6.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2017,3,10,4,1,10,0,0,-11,12,0,0,0,1,0,0,

In [78]:
hist_df.isna().sum()

authorized_flag                0
card_id                        0
city_id                        0
category_1                     0
merchant_category_id           0
merchant_id                    0
purchase_amount                0
state_id                       0
subsector_id                   0
merchant_group_id              0
numerical_1                    0
numerical_2                    0
mer_category_1                 0
most_recent_sales_range        0
most_recent_purchases_range    0
avg_sales_lag3                 0
avg_purchases_lag3             0
avg_sales_lag6                 0
avg_purchases_lag6             0
avg_sales_lag12                0
avg_purchases_lag12            0
category_4                     0
numerical_ratio                0
active_months_lag3_orig        0
active_months_lag6_orig        0
active_months_lag12_orig       0
active_months_lag3_1           0
active_months_lag3_2           0
active_months_lag3_3           0
active_months_lag6_1           0
active_mon

In [148]:
hist_df['price'][hist_df['price']==0]


1167968     0.0
1215551     0.0
10342049    0.0
15588185    0.0
16020320    0.0
18040960    0.0
24844574    0.0
Name: price, dtype: float64

In [ ]:
150000

In [33]:
aggs = {}

aggs['merchant_id'] = ['count', 'nunique', complexity]
aggs['purchase_amount'] = ['sum', 'mean', 'median', 'max', 'min', 'std', Range, mode]
aggs['numerical_1'] = ['sum', 'mean', 'median', 'max', 'min', 'std', Range]
aggs['numerical_2'] = ['sum', 'mean', 'median', 'max', 'min', 'std', Range]
aggs['numerical_ratio'] = ['mean', num_one, num_more_than_one, num_less_than_one, ratio_one, ratio_more_than_one, ratio_less_than_one]

aggs['merchant_group_id'] = ['nunique', num_zero, ratio_zero, Mode, complexity]
aggs['merchant_category_id'] = ['nunique', Mode, complexity]
aggs['subsector_id'] = ['nunique', Mode, complexity]
aggs['state_id'] = ['nunique', Mode, complexity]

aggs['installments_orig'] = ['nunique', num_one, num_zero, num_minusone, num_triplenine, ratio_one, ratio_zero, ratio_minusone, ratio_triplenine, complexity]
aggs['price'] = ['sum','mean','median','max', 'min', 'std', Range]

aggs['most_recent_purchases_range'] = ['sum', 'max', Range, Mode, num_minusone, ratio_minusone, complexity]
aggs['most_recent_sales_range'] = ['sum', 'max', Range, Mode, num_minusone, ratio_minusone, complexity]
aggs['city_id'] = ['nunique', Mode, num_minusone, ratio_minusone, complexity]

aggs['month_lag_orig'] = [num_zero, num_minusone, ratio_zero, ratio_minusone]
aggs['month_diff_orig'] = [ 'max', 'min', Range, Mode, complexity]
                   
## can only be 2017 or 2018
aggs['purchase_date_year'] = ['mean']

for col in ['active_months_lag3_orig', 'active_months_lag6_orig', 'active_months_lag12_orig']:
    aggs[col] = ['nunique', 'median', complexity, Range]

'''
# dont' aggregate these, these are to be grouped
for col in ['purchase_date_weekofyear', 'purchase_date_hour', 'purchase_date_dayofweek',
            'purchase_date_day', 'purchase_date_month', 
            ]:
    aggs[col] = ['nunique', 'median', complexity, Range]

aggs['card_id'] = ['nunique']

'''

# binary_cols values are 0 or 1
## num_one, num_zero, ratio_one, ratio_zero, complexity
## Attention that authorized_flag only exhist in hist_df

binary_cols = ['authorized_flag', 'category_1', 'purchase_date_weekday', 
               'month_lag_-13', 'month_lag_-12',
               'month_lag_-11', 'month_lag_-10', 'month_lag_-9', 'month_lag_-8',
               'month_lag_-7', 'month_lag_-6', 'month_lag_-5', 'month_lag_-4',
               'month_lag_-3', 'month_lag_-2', 'month_lag_-1', 'month_lag_0',
               'month_diff_11', 'month_diff_12', 'month_diff_13', 'month_diff_14',
               'month_diff_15', 'month_diff_16', 'month_diff_17', 'month_diff_18',
               'month_diff_19', 'month_diff_20', 'month_diff_21', 'month_diff_22',
               'month_diff_23', 'month_diff_24', 'installments_-1', 'installments_0',
               'installments_1', 'installments_2', 'installments_3', 'installments_4',
               'installments_5', 'installments_6', 'installments_7', 'installments_8',
               'installments_9', 'installments_10', 'installments_11', 'installments_12',
               'installments_999', 'category_2_1.0','category_2_2.0', 'category_2_3.0',
               'category_2_4.0', 'category_2_5.0', 'category_2_nan', 'category_3_A',
               'category_3_B', 'category_3_C', 'category_3_nan']

for col in binary_cols:
    aggs[col] = [num_one, num_zero, complexity]

# tertiary_cols values are 0, 1 or -1
## num_one, num_zero, num_minusone, ratio_one, ratio_zero, ratio_minusone, complexity
tertiary_cols = ['mer_category_1', 'category_4', 'price', 'active_months_lag3_1', 'active_months_lag3_2',
                 'active_months_lag3_3', 'active_months_lag6_1', 'active_months_lag6_2', 'active_months_lag6_3',
                 'active_months_lag6_4', 'active_months_lag6_5', 'active_months_lag6_6', 'active_months_lag12_1',
                 'active_months_lag12_2', 'active_months_lag12_3', 'active_months_lag12_4', 'active_months_lag12_5',
                 'active_months_lag12_6', 'active_months_lag12_7', 'active_months_lag12_8', 'active_months_lag12_9',
                 'active_months_lag12_10', 'active_months_lag12_11', 'active_months_lag12_12', 'mer_category_2_1.0',
                 'mer_category_2_2.0', 'mer_category_2_3.0', 'mer_category_2_4.0', 'mer_category_2_5.0', 
                 'mer_category_2_nan', 'has_last3_months_revenue', 'has_last_month_transaction']

for col in tertiary_cols:
    aggs[col] = [num_one, num_zero, num_minusone, complexity]

numeric_cols = ['avg_sales_lag3', 'avg_sales_lag6', 
                'avg_sales_lag12', 'avg_purchases_lag3', 
                'avg_purchases_lag6', 'avg_purchases_lag12']

for col in numeric_cols:
    aggs[col] = ['nunique', 'sum', 'mean', 'median', 'max', 'min', 'std', Range]

#hist_per_month = hist_df.groupby(['card_id', 'purchase_date_month']).agg(aggs)
#hist_per_month.head(15)


# gonna take a long time

In [172]:
%%time

#test = hist_df.iloc[:100000,:].groupby(['card_id', 'purchase_date_month']).agg(aggs)

hist_permonth = hist_df_copy.groupby(['card_id', 'purchase_date_month']).agg(aggs)

hist_permonth.to_hdf('Elo_fe.h5', 'hist_permonth_raw')


CPU times: user 2d 5h 33min 39s, sys: 48min 47s, total: 2d 6h 22min 27s
Wall time: 2d 7h 31min 51s


In [173]:
hist_permonth.shape

(2563765, 441)

In [181]:
hist_permonth.head()

,card_id,purchase_date_month,merchant_id_count,merchant_id_nunique,merchant_id_complexity,purchase_amount_sum,purchase_amount_mean,purchase_amount_median,purchase_amount_max,purchase_amount_min,purchase_amount_std,purchase_amount_Range,purchase_amount_mode,numerical_1_sum,numerical_1_mean,numerical_1_median,numerical_1_max,numerical_1_min,numerical_1_std,numerical_1_Range,numerical_2_sum,numerical_2_mean,numerical_2_median,numerical_2_max,numerical_2_min,numerical_2_std,numerical_2_Range,numerical_ratio_mean,numerical_ratio_num_one,numerical_ratio_num_more_than_one,numerical_ratio_num_less_than_one,numerical_ratio_ratio_one,numerical_ratio_ratio_more_than_one,numerical_ratio_ratio_less_than_one,merchant_group_id_nunique,merchant_group_id_num_zero,merchant_group_id_ratio_zero,merchant_group_id_Mode,merchant_group_id_complexity,merchant_category_id_nunique,merchant_category_id_Mode,merchant_category_id_complexity,subsector_id_nunique,subsector_id_Mode,subsector_id_complexity,state_id_nunique,state_id_Mode,state_id_complexity,installments_orig_nunique,installments_orig_num_one,installments_orig_num_zero,installments_orig_num_minusone,installments_orig_num_triplenine,installments_orig_ratio_one,installments_orig_ratio_zero,installments_orig_ratio_minusone,installments_orig_ratio_triplenine,installments_orig_complexity,price_num_one,price_num_zero,price_num_minusone,price_complexity,most_recent_purchases_range_sum,most_recent_purchases_range_max,most_recent_purchases_range_Range,most_recent_purchases_range_Mode,most_recent_purchases_range_num_minusone,most_recent_purchases_range_ratio_minusone,most_recent_purchases_range_complexity,most_recent_sales_range_sum,most_recent_sales_range_max,most_recent_sales_range_Range,most_recent_sales_range_Mode,most_recent_sales_range_num_minusone,most_recent_sales_range_ratio_minusone,most_recent_sales_range_complexity,city_id_nunique,city_id_Mode,city_id_num_minusone,city_id_ratio_minusone,city_id_complexity,month_lag_orig_num_zero,month_lag_orig_num_minusone,month_lag_orig_ratio_zero,month_lag_orig_ratio_minusone,month_diff_orig_max,month_diff_orig_min,month_diff_orig_Range,month_diff_orig_Mode,month_diff_orig_complexity,purchase_date_year_mean,active_months_lag3_orig_nunique,active_months_lag3_orig_median,active_months_lag3_orig_complexity,active_months_lag3_orig_Range,active_months_lag6_orig_nunique,active_months_lag6_orig_median,active_months_lag6_orig_complexity,active_months_lag6_orig_Range,active_months_lag12_orig_nunique,active_months_lag12_orig_median,active_months_lag12_orig_complexity,active_months_lag12_orig_Range,authorized_flag_num_one,authorized_flag_num_zero,authorized_flag_complexity,category_1_num_one,category_1_num_zero,category_1_complexity,purchase_date_weekday_num_one,purchase_date_weekday_num_zero,purchase_date_weekday_complexity,month_lag_-13_num_one,month_lag_-13_num_zero,month_lag_-13_complexity,month_lag_-12_num_one,month_lag_-12_num_zero,month_lag_-12_complexity,month_lag_-11_num_one,month_lag_-11_num_zero,month_lag_-11_complexity,month_lag_-10_num_one,month_lag_-10_num_zero,month_lag_-10_complexity,month_lag_-9_num_one,month_lag_-9_num_zero,month_lag_-9_complexity,month_lag_-8_num_one,month_lag_-8_num_zero,month_lag_-8_complexity,month_lag_-7_num_one,month_lag_-7_num_zero,month_lag_-7_complexity,month_lag_-6_num_one,month_lag_-6_num_zero,month_lag_-6_complexity,month_lag_-5_num_one,month_lag_-5_num_zero,month_lag_-5_complexity,month_lag_-4_num_one,month_lag_-4_num_zero,month_lag_-4_complexity,month_lag_-3_num_one,month_lag_-3_num_zero,month_lag_-3_complexity,month_lag_-2_num_one,month_lag_-2_num_zero,month_lag_-2_complexity,month_lag_-1_num_one,month_lag_-1_num_zero,month_lag_-1_complexity,month_lag_0_num_one,month_lag_0_num_zero,month_lag_0_complexity,month_diff_11_num_one,month_diff_11_num_zero,month_diff_11_complexity,month_diff_12_num_one,month_diff_12_num_zero,month_diff_12_complexity,month_diff_13_num_one,month_diff_13_num_zero,month_diff_13_complexity,month_dif

In [175]:
hist_permonth.columns = ['_'.join(col).strip() for col in hist_permonth.columns.values]


In [180]:
hist_permonth = hist_permonth.reset_index()

In [182]:
hist_permonth['purchase_date_month'].value_counts()

12    249694
11    240024
13    230443
10    224727
9     211242
14    205999
8     195913
7     180091
6     163374
5     152839
4     142742
3     134528
2     120965
1     111184
Name: purchase_date_month, dtype: int64

In [ ]:



import sys
import numpy as np
import pandas as pd

input_h5 = sys.argv[1]

hist_df = pd.read_hdf(input_h5, 'hist_df_fillna')
new_merchant_df = pd.read_hdf(input_h5, 'new_merchant_df_fillna')

def Range(x):
    res = max(x) - min(x)
    return res
def num_zero(x):
    return len(x[x==0])
def num_one(x):
    return len(x[x==1])

def num_minusone(x):
    return len(x[x==-1])

def num_triplenine(x):
    return len(x[x==999])
 
def num_more_than_one(x):
    return len(x[x>1])
def num_less_than_one(x):
    return len(x[x<1])
def sumsq(x):
    return sum(x**2)
def ratio_one(x):
    return len(x[x==1.0])/len(x)
def ratio_zero(x):
    return len(x[x==0.0])/len(x)
def ratio_minusone(x):
    return len(x[x==-1.0])/len(x)
def ratio_triplenine(x):
    return len(x[x==999])/len(x)
def ratio_more_than_one(x):
    return len(x[x>1.0])/len(x)
def ratio_less_than_one(x):
    return len(x[x<1.0])/len(x)

def complexity(x):
    counts = np.array(list(Counter(x).values()))
    S = sum(counts)
    counts = counts/S
    return sumsq(counts)
def Mode(x):
    # if the most frequent & 2nd most frequent one have the same frequencies, take the bigger one
    counts = Counter(x)
    tup = sorted(counts.items(), key=lambda kv: kv[1])
    return tup[-1][0]
def mode(x):
    # take average of the 2
    counts = Counter(x)
    tup = sorted(counts.items(), key=lambda kv: kv[1])
    try:
        assert tup[-1][1] == tup[-2][1]
        return (tup[-1][d0] + tup[-2][0])/2.0
    except Exception:
        return tup[-1][0]

aggs = {}
aggs['merchant_id'] = ['count', 'nunique', complexity]
aggs['purchase_amount'] = ['sum', 'mean', 'median', 'max', 'min', 'std', Range, mode]
aggs['numerical_1'] = ['sum', 'mean', 'median', 'max', 'min', 'std', Range]
aggs['numerical_2'] = ['sum', 'mean', 'median', 'max', 'min', 'std', Range]
aggs['numerical_ratio'] = ['mean', ratio_one, ratio_more_than_one, ratio_less_than_one]
aggs['merchant_group_id'] = ['nunique', num_zero, Mode, complexity]
aggs['merchant_category_id'] = ['nunique', Mode, complexity]
aggs['subsector_id'] = ['nunique', Mode, complexity]
aggs['state_id'] = ['nunique', Mode, complexity]
aggs['installments_orig'] = ['nunique', num_one, num_zero, num_minusone, num_triplenine, ratio_one, ratio_zero, ratio_minusone, complexity]
aggs['price'] = ['sum','mean','median','max', 'min', 'std', Range]
aggs['most_recent_purchases_range'] = ['sum', 'max', Range, Mode, num_minusone, ratio_minusone, complexity]
aggs['most_recent_sales_range'] = ['sum', 'max', Range, Mode, num_minusone, ratio_minusone, complexity]
aggs['city_id'] = ['nunique', Mode, complexity]
aggs['month_lag_orig'] = [num_zero, num_minusone, ratio_zero, ratio_minusone]
aggs['month_diff_orig'] = [ 'max', 'min', Range, Mode, complexity]            
## can only be 2017 or 2018
aggs['purchase_date_year'] = ['mean']

for col in ['active_months_lag3_orig', 'active_months_lag6_orig', 'active_months_lag12_orig']:
    aggs[col] = ['nunique', 'median', complexity, Range]

binary_cols = ['authorized_flag', 'category_1', 'purchase_date_weekday', 
               'month_lag_-13', 'month_lag_-12',
               'month_lag_-11', 'month_lag_-10', 'month_lag_-9', 'month_lag_-8',
               'month_lag_-7', 'month_lag_-6', 'month_lag_-5', 'month_lag_-4',
               'month_lag_-3', 'month_lag_-2', 'month_lag_-1', 'month_lag_0',
               'month_diff_11', 'month_diff_12', 'month_diff_13', 'month_diff_14',
               'month_diff_15', 'month_diff_16', 'month_diff_17', 'month_diff_18',
               'month_diff_19', 'month_diff_20', 'month_diff_21', 'month_diff_22',
               'month_diff_23', 'month_diff_24', 'installments_-1', 'installments_0',
               'installments_1', 'installments_2', 'installments_3', 'installments_4',
               'installments_5', 'installments_6', 'installments_7', 'installments_8',
               'installments_9', 'installments_10', 'installments_11', 'installments_12',
               'installments_999', 'category_2_1.0','category_2_2.0', 'category_2_3.0',
               'category_2_4.0', 'category_2_5.0', 'category_2_nan', 'category_3_A',
               'category_3_B', 'category_3_C', 'category_3_nan']

for col in binary_cols:
    aggs[col] = [num_one, num_zero, complexity]

tertiary_cols = ['mer_category_1', 'category_4', 'price', 'active_months_lag3_1', 'active_months_lag3_2',
                 'active_months_lag3_3', 'active_months_lag6_1', 'active_months_lag6_2', 'active_months_lag6_3',
                 'active_months_lag6_4', 'active_months_lag6_5', 'active_months_lag6_6', 'active_months_lag12_1',
                 'active_months_lag12_2', 'active_months_lag12_3', 'active_months_lag12_4', 'active_months_lag12_5',
                 'active_months_lag12_6', 'active_months_lag12_7', 'active_months_lag12_8', 'active_months_lag12_9',
                 'active_months_lag12_10', 'active_months_lag12_11', 'active_months_lag12_12', 'mer_category_2_1.0',
                 'mer_category_2_2.0', 'mer_category_2_3.0', 'mer_category_2_4.0', 'mer_category_2_5.0', 
                 'mer_category_2_nan', 'has_last3_months_revenue', 'has_last_month_transaction']

for col in tertiary_cols:
    aggs[col] = [num_one, num_zero, num_minusone, complexity]

numeric_cols = ['avg_sales_lag3', 'avg_sales_lag6', 
                'avg_sales_lag12', 'avg_purchases_lag3', 
                'avg_purchases_lag6', 'avg_purchases_lag12']

for col in numeric_cols:
    aggs[col] = aggs[col] = ['sum', 'std']
    
hist_permonth = hist_df.groupby(['card_id', 'purchase_date_month']).agg(aggs)
print('HIST GROUPBY COMPLETE.....')
print('\t.............\t')
hist_permonth.to_hdf('hist_permonth.h5', 'hist_permonth')
print('CONVERTED INTO hist_permonth.h5')
print('\t.............\t')



aggs = {}
aggs['merchant_id'] = ['count', 'nunique', complexity]
aggs['purchase_amount'] = ['sum', 'mean', 'median', 'max', 'min', 'std', Range, mode]
aggs['numerical_1'] = ['sum', 'mean', 'median', 'max', 'min', 'std', Range]
aggs['numerical_2'] = ['sum', 'mean', 'median', 'max', 'min', 'std', Range]
aggs['numerical_ratio'] = ['mean', ratio_one, ratio_more_than_one, ratio_less_than_one]
aggs['merchant_group_id'] = ['nunique', num_zero, Mode, complexity]
aggs['merchant_category_id'] = ['nunique', Mode, complexity]
aggs['subsector_id'] = ['nunique', Mode, complexity]
aggs['state_id'] = ['nunique', Mode, complexity]
aggs['installments_orig'] = ['nunique', num_one, num_zero, num_minusone, num_triplenine, ratio_one, ratio_zero, ratio_minusone, complexity]
aggs['price'] = ['sum','mean','median','max', 'min', 'std', Range]
aggs['most_recent_purchases_range'] = ['sum', 'max', Range, Mode, num_minusone, ratio_minusone, complexity]
aggs['most_recent_sales_range'] = ['sum', 'max', Range, Mode, num_minusone, ratio_minusone, complexity]
aggs['city_id'] = ['nunique', Mode, complexity]
aggs['month_lag_orig'] = [num_zero, num_minusone, ratio_zero, ratio_minusone]
aggs['month_diff_orig'] = [ 'max', 'min', Range, Mode, complexity]            
## can only be 2017 or 2018
aggs['purchase_date_year'] = ['mean']

for col in ['active_months_lag3_orig', 'active_months_lag6_orig', 'active_months_lag12_orig']:
    aggs[col] = ['nunique', 'median', complexity, Range]


# dont' aggregate these, these are to be grouped
for col in ['purchase_date_weekofyear', 'purchase_date_hour', 'purchase_date_dayofweek',
            'purchase_date_day', 'purchase_date_month', 
            ]:
    aggs[col] = ['nunique', 'median', complexity, Range]

aggs['card_id'] = ['nunique']



binary_cols = ['category_1', 'purchase_date_weekday', 'month_lag_1', 'month_lag_2',
               'month_diff_11', 'month_diff_12', 'month_diff_13', 'month_diff_14',
               'month_diff_15', 'month_diff_16', 'month_diff_17', 'month_diff_18',
               'month_diff_19', 'month_diff_20', 'month_diff_21', 'month_diff_22',
               'month_diff_23', 'month_diff_24', 'installments_-1', 'installments_0',
               'installments_1', 'installments_2', 'installments_3', 'installments_4',
               'installments_5', 'installments_6', 'installments_7', 'installments_8',
               'installments_9', 'installments_10', 'installments_11', 'installments_12',
               'installments_999', 'category_2_1.0','category_2_2.0', 'category_2_3.0',
               'category_2_4.0', 'category_2_5.0', 'category_2_nan', 'category_3_A',
               'category_3_B', 'category_3_C', 'category_3_nan']

for col in binary_cols:
    aggs[col] = [num_one, num_zero, complexity]

# tertiary_cols values are 0, 1 or -1
tertiary_cols = ['mer_category_1', 'category_4', 'price', 'active_months_lag3_1', 'active_months_lag3_2',
                 'active_months_lag3_3', 'active_months_lag6_1', 'active_months_lag6_2', 'active_months_lag6_3',
                 'active_months_lag6_4', 'active_months_lag6_5', 'active_months_lag6_6', 'active_months_lag12_1',
                 'active_months_lag12_2', 'active_months_lag12_3', 'active_months_lag12_4', 'active_months_lag12_5',
                 'active_months_lag12_6', 'active_months_lag12_7', 'active_months_lag12_8', 'active_months_lag12_9',
                 'active_months_lag12_10', 'active_months_lag12_11', 'active_months_lag12_12', 'mer_category_2_1.0',
                 'mer_category_2_2.0', 'mer_category_2_3.0', 'mer_category_2_4.0', 'mer_category_2_5.0', 
                 'mer_category_2_nan', 'has_last3_months_revenue', 'has_last_month_transaction']

for col in tertiary_cols:
    aggs[col] = [num_one, num_zero, num_minusone, complexity]

numeric_cols = ['avg_sales_lag3', 'avg_sales_lag6', 
                'avg_sales_lag12', 'avg_purchases_lag3', 
                'avg_purchases_lag6', 'avg_purchases_lag12']

for col in numeric_cols:
    aggs[col] = ['sum', 'std']


new_permonth = new_merchant_df.groupby(['card_id', 'purchase_date_month']).agg(aggs)
print('NEW GROUPBY COMPLETE.....')
print('\t.............\t')
new_permonth.to_hdf('new_permonth.h5', 'new_permonth')
print('CONVERTED INTO new_permonth.h5')
print('\t.............\t')


In [ ]:
#merchants.to_hdf('Elo_fe.h5', 'merchants')


In [151]:
hist_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29208684 entries, 0 to 29208683
Columns: 120 entries, authorized_flag to price
dtypes: float64(46), int64(19), object(2), uint8(53)
memory usage: 17.5+ GB


In [152]:
hist_df_copy = hist_df.copy()

In [154]:
for col in hist_df_copy.columns:
    if hist_df_copy[col].dtypes == 'float':
        hist_df_copy[col] = (hist_df_copy[col]).astype('float32')

In [162]:
for col in hist_df_copy.columns:
    if hist_df_copy[col].dtypes == 'int':
        hist_df_copy[col] = (hist_df_copy[col]).astype('int32')

In [165]:
hist_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29208684 entries, 0 to 29208683
Columns: 120 entries, authorized_flag to price
dtypes: float32(46), int32(19), object(2), uint8(53)
memory usage: 9.2+ GB


In [160]:
%%time
test = hist_df.iloc[:10000,:].groupby(['card_id', 'purchase_date_month']).agg(aggs)


CPU times: user 31 s, sys: 324 ms, total: 31.4 s
Wall time: 31.3 s


In [161]:
%%time
test = hist_df_copy.iloc[:10000,:].groupby(['card_id', 'purchase_date_month']).agg(aggs)


CPU times: user 30.6 s, sys: 294 ms, total: 30.9 s
Wall time: 30.8 s


In [163]:
len(hist_df_copy)

29208684

In [164]:
29208684/10000

2920.8684

In [166]:
del new_merchant_df
gc.collect()

1018

In [169]:
test.shape

(402, 441)

In [171]:
hist_df_copy.iloc[:10000,:].shape

(10000, 120)